In [ ]:
from ovotools.params import AttrDict
import sys
sys.path.append('..')
sys.path.append('../NN/RetinaNet')
import local_config
from os.path import join
model_name = 'NN_results/retina_points_cde773'
model_fn = join(local_config.data_path, model_name)

params = AttrDict.load(model_fn + '.param.txt', verbose = True)
params.data.net_hw = (128,128,) #(512,768) ###### (1024,1536) #
params.data.batch_size = 1 #######

params.data.get_points = False
#params.model_params.encoder_params.anchor_areas=[5 * 5., ] # 6 * 6., 10 * 10.,
params.model_params.encoder_params.iuo_fit_thr = 0 # if iou > iuo_fit_thr => rect fits anchor
params.model_params.encoder_params.iuo_nofit_thr = 0

params.augmentation = AttrDict(
    img_width_range=(1100, 1100),
    stretch_limit = 0.1,
    rotate_limit=5,
)

import torch
import ignite

device = 'cuda:0'


# In[3]:

import DSBI_invest.data
import create_model_retinanet
from data import int_to_label


model, collate_fn, loss = create_model_retinanet.create_model_retinanet(params, phase='train', device=device)
model = None

train_loader, (val_loader1, val_loader2) = DSBI_invest.data.create_dataloaders(params, collate_fn,
                                                                               mode = 'debug', verbose = 2)
val_loader1_it = iter(val_loader1)

import PIL
import PIL.ImageDraw
import PIL.ImageFont
import numpy as np
def TensorToPilImage(tensor, params):
    std = (0.2,) #params.data.std
    mean = (0.5,) #params.data.mean
    vx_np = tensor.cpu().numpy().copy()
    vx_np *= np.asarray(std)[:, np.newaxis, np.newaxis]
    vx_np += np.asarray(mean)[:, np.newaxis, np.newaxis]
    vx_np = vx_np.transpose(1,2,0)*255
    return PIL.Image.fromarray(vx_np.astype(np.uint8))

encoder = loss.encoder

In [ ]:
#val_loader1_it = iter(val_loader1)
batch = next(val_loader1_it)
data, target = ignite.engine._prepare_batch(batch, device=device)
TensorToPilImage(data[0], params)

In [ ]:
cls_thresh = 0.6
nms_thresh = 0.

img = TensorToPilImage(data[0], params)
w,h = img.size

labels = target[1][0].clamp(min=0)
ly = torch.eye(65, device = labels.device)  # [D,D]
cls_preds = ly[labels][:,1:]

boxest, labelst = encoder.decode(target[0][0].cpu().data, cls_preds, (w,h),
                              cls_thresh = cls_thresh, nms_thresh = nms_thresh)
print(len(boxest))
import PIL.ImageFont
draw = PIL.ImageDraw.Draw(img)
fnt = PIL.ImageFont.truetype("arial.ttf", 8)
for i,box in enumerate(boxest):
    draw.rectangle(list(box), outline='green')
    draw.text((box[0],box[3]), int_to_label(labelst[i].item()), font=fnt, fill=(255,255,255,128))
img